In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path

#import api_key
#from api_keys import api_keys

In [12]:
#Createing path to CSV files
Flu_path = Path("../Resources/ILINet.csv")
Temp_path = Path("../Resources/TempByState_1855_2013.csv")


In [14]:
#Create data frame
Flu_DF = pd.read_csv(Flu_path)
Temp_DF = pd.read_csv(Temp_path)

Flu_DF.head()
Temp_DF.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [15]:
Flu_DF.head()

PERCENTAGE OF VISITS FOR INFLUENZA-LIKE-ILLNESS REPORTED BY SENTINEL PROVIDERS
REGION TYPE REGION   YEAR WEEK % WEIGHTED ILI %UNWEIGHTED ILI AGE 0-4 AGE 25-49 AGE 25-64 AGE 5-24 AGE 50-64 AGE 65 ILITOTAL NUM. OF PROVIDERS                                     TOTAL PATIENTS                            
HHS Regions Region 4 2016 40   1.22375        1.36456         637     369       NaN       801      115       84     2006     306                                                           147007                            
                          41   1.23174        1.45288         714     399       NaN       769      153       96     2131     317                                                           146674                            
                          42   1.2768         1.51243         782     414       NaN       898      185       93     2372     323                                                           156834                            
                          43   1.38032        1.66763         789     532       NaN       994      198       109    2622     325                                                           157229

In [17]:
Temp_df_us = pd.DataFrame(Temp_DF.loc[Temp_DF["Country"] == "United States",:])

Temp_df_us.head()


,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
7458,1743-11-01,10.722,2.898,Alabama,United States
7459,1743-12-01,NaN,NaN,Alabama,United States
7460,1744-01-01,NaN,NaN,Alabama,United States
7461,1744-02-01,NaN,NaN,Alabama,United States
7462,1744-03-01,NaN,NaN,Alabama,United States
